In [6]:
import json
import math
from tqdm import tqdm
from evaluation_metrices import Evaluator
    
training_dataset_ground_truth_file_path = "datasets/final_correct_datasets/training/parsed_data_final.json"
training_dataset_predicted_file_path = 'Traditional_IR/Results/bm25_training_results.json'

In [7]:
import json

def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        return data


In [9]:
ground_truth_data = load_json(training_dataset_ground_truth_file_path)
predicted_data = load_json(training_dataset_predicted_file_path)

evaluator = Evaluator(ground_truth_data=ground_truth_data, predicted_data=predicted_data)
results = evaluator.evaluate_metrics_for_articles(k=10)

print(results)

Processing questions...: 5390it [00:00, 97913.29it/s]

{'MRR': 0.8120547309833032, 'MAP': 0.5138563827722564, 'nDCG@10': 0.695807757744625, 'count': 5390}
